In [1]:
import requests
import os
from dotenv import load_dotenv
from array import array
from PIL import Image
import sys
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

In [2]:
load_dotenv()

True

In [3]:
url = "https://invoiceocrp3.azurewebsites.net/invoices"

headers = {
    "accept": "application/json"
}

In [4]:
data = []

In [5]:
response = requests.get(url, headers=headers)
data = response.json()
data = data.get('invoices')

In [6]:
lastDate = data[-1]['dt']
flag = False
while not flag : 
    urldata  = f"https://invoiceocrp3.azurewebsites.net/invoices?start_date={lastDate}"
    responseData = requests.get(urldata, headers=headers)
    if responseData.status_code == 200:
            adddata = responseData.json()
            adddata = adddata.get('invoices')
            if len(adddata) == 0:
                flag = True
            else:
                data.extend(adddata)
                lastDate = data[-1]['dt']
                print(lastDate,data[-1])
    else :
        print(f"Erreur: {response.status_code} - {response.reason}") 
        flag = True

2020-10-18 05:00:00 {'no': 'FAC_2020_0823-2487', 'dt': '2020-10-18 05:00:00'}
2021-07-24 00:09:00 {'no': 'FAC_2021_0641-519400', 'dt': '2021-07-24 00:09:00'}
2022-05-26 23:53:00 {'no': 'FAC_2022_0474-6057400', 'dt': '2022-05-26 23:53:00'}
2023-03-28 16:10:00 {'no': 'FAC_2023_0295-359724', 'dt': '2023-03-28 16:10:00'}
2024-02-08 06:03:00 {'no': 'FAC_2024_0122-4602818', 'dt': '2024-02-08 06:03:00'}
2024-03-24 23:21:00 {'no': 'FAC_2024_0282-2533050', 'dt': '2024-03-24 23:21:00'}


In [8]:
print(data[5]['no'])
print(len(data))

FAC_2019_0005-1869518
6165


In [9]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [10]:

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = f"https://invoiceocrp3.azurewebsites.net/invoices/{data[5]['no']}"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

count = 0
# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

===== Read File - remote =====


In [32]:
# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines: 
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - remote
'''

print("End of Computer Vision quickstart.")

===== Read File - remote =====
INVOICE FAC_2019_0005
[18.0, 19.0, 303.0, 19.0, 303.0, 43.0, 18.0, 42.0]
2
Issue date 2019-01-03 22:10:00
[21.0, 49.0, 309.0, 48.0, 309.0, 70.0, 21.0, 71.0]
4
Bill to Victoria Parker
[19.0, 80.0, 198.0, 80.0, 198.0, 99.0, 19.0, 98.0]
6
Brilllling
[738.0, 94.0, 809.0, 75.0, 812.0, 91.0, 742.0, 112.0]
8
Address USCGC Park
[17.0, 119.0, 132.0, 119.0, 132.0, 132.0, 17.0, 131.0]
10
FPO AE 88629
[18.0, 132.0, 100.0, 133.0, 100.0, 145.0, 18.0, 145.0]
12
Dolores amet et natus.
[39.0, 200.0, 233.0, 200.0, 233.0, 219.0, 39.0, 219.0]
14
4 x
[510.0, 201.0, 552.0, 203.0, 551.0, 218.0, 510.0, 217.0]
16
59.13 Euro
[567.0, 201.0, 691.0, 202.0, 690.0, 219.0, 567.0, 217.0]
18
Reprehenderit quae veritatis enim.
[39.0, 220.0, 332.0, 220.0, 332.0, 241.0, 39.0, 240.0]
20
9 x
[510.0, 221.0, 552.0, 223.0, 553.0, 239.0, 510.0, 238.0]
22
76.39 Euro
[569.0, 220.0, 690.0, 222.0, 689.0, 239.0, 568.0, 237.0]
24
Fugit voluptatum fuga molestiae.
[39.0, 240.0, 322.0, 240.0, 322.0, 260.0,

In [ ]:
text_result.lines

In [39]:
donnees_lignes = { 'facture' : 
    {"text": ligne.text, "bounding_box": ligne.bounding_box}
    for ligne in  text_result.lines
}

In [41]:
donnees_lignes = {}

# Itérez à travers chaque objet de ligne avec un index
for index, ligne_objet in enumerate( text_result.lines):
    # Créez un dictionnaire pour stocker les données de cette ligne
    donnees_ligne = {
        "text": ligne_objet.text,
        "bounding_box": ligne_objet.bounding_box,
        # Ajoutez plus d'attributs selon vos besoins
    }
    
    # Utilisez l'index comme identifiant unique pour chaque ligne
    identifiant_ligne = f"ligne_{index}"
    
    # Stockez les données de cette ligne dans le dictionnaire principal avec l'identifiant comme clé
    donnees_lignes[identifiant_ligne] = donnees_ligne

# Maintenant, donnees_lignes est un dictionnaire où chaque clé est un identifiant de ligne unique et chaque valeur est un dictionnaire contenant les données de la ligne correspondante

In [ ]:
donnees_lignes

In [34]:
for row in range(len(data)) :
    '''
    OCR: Read File using the Read API, extract text - remote
    This example will extract text in an image, then print results, line by line.
    This API call can also extract handwriting style text (not shown).
    '''
    # Get an image with text
    read_image_url = f"https://invoiceocrp3.azurewebsites.net/invoices/{data[row]['no']}"

    # Call API with URL and raw response (allows you to get the operation location)
    read_response = computervision_client.read(read_image_url,  raw=True)

    # Get the operation location (URL with an ID at the end) from the response
    read_operation_location = read_response.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = read_operation_location.split("/")[-1]

    count = 0
    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    facture_dict = {}
    facture = { data[row]['no']: []}
    facture_dict.append(facture)
    print(facture)
    """if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
    # Itérez à travers chaque objet de ligne
            for ligne_objet in text_result.lines:
            # Créez un dictionnaire pour stocker les données de cette ligne
                    donnees_ligne = {
                        "text": ligne_objet.text,
                        "bounding_box": ligne_objet.bounding_box,
                        # Ajoutez plus d'attributs selon vos besoins
                    }
                    
                    # Ajoutez les données de cette ligne à la liste correspondante dans le dictionnaire principal
                    print(donnees_ligne)
                    facture[data[row]['no']].append(donnees_ligne)"""
    

            # Maintenant, donnees_lignes contient un dictionnaire avec une seule clé "toutes_les_lignes" qui contient une liste de tous les dictionnaires de données de ligne

{'FAC_2019_0002-521208': []}
{'FAC_2019_0001-112650': []}
{'FAC_2019_0003-4174848': []}


KeyboardInterrupt: 

In [35]:
facture

{'FAC_2019_0003-4174848': []}